In [4]:
import subprocess
import os
# 로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행하세요
# 그 후에 다시 주석 해제해주세요.
user = "root"
host = "172.16.8.11"
password = "asd123!@"


########################### 로컬 ###########################
## 우선 로컬에서 원격 호스트 IP에 대한 StrictHostKeyChecking=no를 항상 허용해줍니다.

def local_ssh_config():
    ssh_dir = os.path.expanduser("~/.ssh")
    config_path = os.path.join(ssh_dir,"config")
    
    os.makedirs(ssh_dir, exist_ok=True)
    os.chmod(ssh_dir, 0o700)
    
    config_content = f"""
    Host {host}
        StrictHostKeyChecking no
        UserKnownHostsFile=/dev/null
    """
    
    # 이미 같은 설정이 있는지 확인하고 없으면 추가
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            content = f.read()
    else:
        content = ""
    
    if host not in content:
        with open(config_path, "a") as f:
            f.write(config_content)
        print(f"✅ SSH 설정 추가 완료: {host}")
    else:
        print(f"ℹ️ 이미 설정되어 있습니다: {host}")
    
    # 권한 설정 (보안상 필요)
    os.chmod(config_path, 0o600)

def local_set():
    print("로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행")
    os.system("dnf install -y expect")

#local_set()

########################### 원격 ###########################

def cc(command):
    cmd = f"sshpass -p '{password}' ssh {user}@{host} '{command}'"
    result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
    print(result.stdout)

def install_packages():
    # 여러 명령을 한 줄로 연결하기 (&& 로 순차 실행)
    commands = """
     dnf -y install net-tools &&
     dnf -y install vim &&
     dnf install -y epel-release &&
     dnf -y update && dnf -y upgrade
     dnf -y install wget
     dnf -y install expect
    """
    cc(commands)

def install_miniconda_and_config():
    print("Miniconda를 설치합니다.")
    print("/etc/profile.d/conda.sh 파일에 Miniconda를 등록해서 , 터미널 어디서든 conda 명령을 쓸 수 있게 만드는 설정입니다.")
    commands = """
cd ~
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
bash miniconda.sh -b -p /opt/miniconda3
echo 'export PATH="/opt/miniconda3/bin:$PATH"' >> /etc/profile.d/conda.sh
source /etc/profile.d/conda.sh
"""
    cc(commands)

def install_conda_and_paramiko():
    print("conda를 설치합니다")
    commands = """
/opt/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
/opt/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
/opt/miniconda3/bin/conda install -y jupyter
"""
    cc("conda install -c conda-forge types-cryptography")
    cc("python -m pip install paramiko")
    

def set_firewalld_and_selinux_off():
    print("firewalld , selinux 방화벽을 끕니다")
    cc("systemctl stop firewalld")
    commands = f"""
sudo sed -i 's/^SELINUX=.*/SELINUX=disabled/' /etc/selinux/config
"""
    cc(commands)

def set_jupyter_passwd():
    passwd = input("주피터 패스워드 비밀번호를 설정하세요 (ex. asd123!@)")
    commands = f'''
sudo cat << EOF > /root/jupyter_passwd.exp
#!/usr/bin/expect -f
spawn /opt/miniconda3/bin/jupyter server password
expect "Enter password:"
send "{passwd}\\r"
expect "Verify password:"
send "{passwd}\\r"
expect eof
EOF

sudo chmod +x /root/jupyter_passwd.exp
sudo /usr/bin/expect /root/jupyter_passwd.exp
'''
    cc(commands)


# 재부팅 시 자동 활성화 설정입니다.
def set_systemd_service():
    commands = f"""
sudo cat << EOF > /etc/systemd/system/jupyter.service
[Unit]
Description=Jupyter Notebook
After=network.target

[Service]
Type=simple
User=root
# 설치 경로를 /opt로 변경했으므로 WorkingDirectory도 /opt로 설정하는 것이 좋음
WorkingDirectory=/opt
# ExecStart의 모든 경로를 /opt/miniconda3 기준으로 수정
ExecStart=/opt/miniconda3/bin/jupyter notebook --config=/root/.jupyter/jupyter_server_config.json --ip={host} --port=8000 --no-browser --allow-root

[Install]
WantedBy=multi-user.target
EOF

sudo systemctl daemon-reload
sudo systemctl enable jupyter
sudo systemctl start jupyter
sudo systemctl status jupyter
"""
    cc(commands)

# 실행
def install_in():
    install_packages()
    install_miniconda_and_config()   
    install_conda_and_paramiko()
    set_firewalld_and_selinux_off()
    set_jupyter_passwd()
    set_systemd_service()
    
install_in()


● jupyter.service - Jupyter Notebook
     Loaded: loaded (/etc/systemd/system/jupyter.service; enabled; preset: disabled)
     Active: active (running) since Tue 2025-11-11 18:09:46 KST; 50ms ago
   Main PID: 59774 (jupyter)
      Tasks: 1 (limit: 22933)
     Memory: 5.1M
        CPU: 24ms
     CGroup: /system.slice/jupyter.service
             └─59774 /opt/miniconda3/bin/python /opt/miniconda3/bin/jupyter notebook --config=/root/.jupyter/jupyter_server_config.json --ip=172.16.8.11 --port=8000 --no-browser --allow-root

Nov 11 18:09:46 localhost.localdomain systemd[1]: Started Jupyter Notebook.

